In [1]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM
from nltk.tokenize import sent_tokenize
import nltk
import time
import pickle
from torch.utils.data import DataLoader

path = "/global/cscratch1/sd/ajaybati/pickles/"


In [23]:
noteevents = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/NOTEEVENTS.csv")


final_df = pd.read_pickle(path+"final_df.pickle")
notes_df = noteevents[["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME","CATEGORY","TEXT"]]
notes_df = final_df[["HADM_ID","SUBJECT_ID","DOB"]].drop_duplicates().merge(notes_df, on=["SUBJECT_ID","HADM_ID"], how="right")

notes = noteevents[["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME","CATEGORY","TEXT"]]
notes_df = pd.merge(final_df[["SUBJECT_ID","HADM_ID","DOB"]],notes.drop_duplicates(subset = ["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME"]), on=["SUBJECT_ID","HADM_ID"], how = "left")
notes_df = notes_df.drop_duplicates(subset='TEXT')
notes_df

salience_patient = pd.read_pickle(path+"salience_patient.pickle")



/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#data collection
import random
class textLoader(Dataset):
    def __init__(self,transform = None, yes = True):
        self.input_ids_all = []
        self.attention_masks_all = []
        self.text = notes_df["TEXT"].tolist()
        self.samples = len(self.text)
        total = 0
        x=0
        for note in self.text[250000:]:
            start = time.time()
            try:
                for sent in sent_tokenize(note):
                    
                    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,
                        truncation = True,# Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
                    # Add the encoded sentence to the list.    
                    input_ids = encoded_dict['input_ids']
                    attention_masks = encoded_dict['attention_mask']

                    if yes:
                        input_ids_real = []
                        for index,word in enumerate(input_ids[0]):
                            if int(word) != 101 and int(word) != 102:
                                rando = random.random()
                                random.seed()
                                if rando < 0.2 and int(word)!=0:
                                    input_ids_real.append(103)
                                else:
                                    input_ids_real.append(int(word))
                            else:
                                input_ids_real.append(int(word))
                        input_ids_real = torch.tensor(input_ids_real).view(1,64)

                        self.input_ids_all.append(input_ids_real)
                        self.attention_masks_all.append(attention_masks)
                    else:

                        self.input_ids_all.append(input_ids)
                        self.attention_masks_all.append(attention_masks)
                        
            except Exception as e:
                print(str(e), input_ids)
            end = time.time()
            if((self.text.index(note)+1)%10000==0):
                x+=1
                total+=(end-start)
                print((self.text.index(note)+1)/367101+0.6810114927499517, total/(x))
       
            
    def __getitem__(self,index):
        return self.input_ids_all[index],self.attention_masks_all[index]
    
    
    def __len__(self):
        return self.samples

    
print("="*20,"data randomized 0.6810114927499517+","="*20)

data_2 = textLoader(yes = True)
print("done")
data_input_randomized = torch.cat(tuple(data_2.input_ids_all),dim = 0)
attention_masks_randomized = torch.cat(tuple(data_2.attention_masks_all),dim = 0)
print("save 1")
torch.save(data_input_randomized,"pickles/input_ids_randomized30.pickle")
print("save 2")
torch.save(attention_masks_randomized,"pickles/attention_masks_randomized30.pickle")
print("done")

In [3]:
input_ids_randomized70 = torch.load(path+"input_ids_randomized70.pickle")
attention_masks_randomized70 = torch.load(path+"attention_masks_randomized70.pickle")
REDOinput_ids_randomized30 = torch.load(path+"REDOinput_ids_randomized30.pickle")
REDOattention_masks_randomized30 = torch.load(path+"REDOattention_masks_randomized30.pickle")
print(len(input_ids_randomized70)+len(REDOinput_ids_randomized30))

8119516


In [4]:
#preprocessing data before save
fullist2 = list(attention_masks_randomized70)
for x in REDOattention_masks_randomized30:
    fullist2.append(x)


fullist = list(input_ids_randomized70)
for x in REDOinput_ids_randomized30:
    fullist.append(x)


fullist_changed2 = []
for tens in fullist2:
    tens = tens.view(1,64)
    fullist_changed2.append(tens)
fullist_changed2

fullist_changed = []
for tens in fullist:
    tens = tens.view(1,64)
    fullist_changed.append(tens)
fullist_changed

attention_masks_randomized100 = torch.cat(tuple(fullist_changed2),0)
len(attention_masks_randomized100)

input_ids_randomized100 = torch.cat(tuple(fullist_changed),0)
len(input_ids_randomized100)



8119516

In [3]:
input_ids_all = torch.load(path+"input_ids_randomized100.pickle")
attention_masks_all = torch.load(path+"attention_masks_randomized100.pickle")
input_ids_real = torch.load(path+"input_ids_real.pickle")

In [4]:
print(input_ids_all[5501010])
print(input_ids_real[5501010])

tensor([  101, 21442, 26915,  6633,  2318,   103,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor([  101, 21442, 26915,  6633,  2318,  2651,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0

In [5]:
# mask_indices = []
# x=0
# for sent in input_ids_all:
#     start=time.time()
#     sentence = list(sent)
#     mask = []
#     for word in sentence:
#         if word==103:
#             mask.append(index)
#     mask_indices.append(mask)
#     end = time.time()
#     x+=1
#     if x%500000==0:
#         print("="*20+" "+str(x)+"/8286324"+"="*20, len(mask)/len(masked_indices))
with open(path+"mask_indices.pickle","rb") as f:
    mask_indices = pickle.load(f)

In [6]:
mask_indices_tensor = []
number=0
for mask in mask_indices:
    number+=1
    padded = []
    try:
        a = time.time()
        if len(mask)==0:
            for x in range(40):
                padded.append(0)
        else:
            padded = padded+mask
            for x in range(40-len(padded)):
                padded.append(0)
        mask_indices_tensor.append(torch.tensor(padded).view(1,40))
        if ((number%1000000)==0):
            print("="*20+str(number)+"/8119516"+"="*20)
            print(time.time()-a)
            
    except Exception as e:
        print(str(e))
        print(type(mask))
        print(padded)
        time.sleep(100)
mask_indices_tensor = torch.cat(tuple(mask_indices_tensor),0)
mask_indices_tensor

====================1000000/8119516====================
0.00010085105895996094
====================2000000/8119516====================
0.00011396408081054688
====================3000000/8119516====================
0.00010228157043457031
====================4000000/8119516====================
0.0001289844512939453
====================5000000/8119516====================
0.00010800361633300781
====================6000000/8119516====================
0.0002570152282714844
====================7000000/8119516====================
0.00011968612670898438
====================8000000/8119516====================
0.00023818016052246094


tensor([[ 3,  6, 24,  ...,  0,  0,  0],
        [ 6,  8, 10,  ...,  0,  0,  0],
        [ 6, 13,  0,  ...,  0,  0,  0],
        ...,
        [ 8, 13, 17,  ...,  0,  0,  0],
        [ 4,  7,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]])

In [7]:
len(mask_indices_tensor)

8119516

In [8]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids_all, attention_masks_all, input_ids_real, mask_indices_tensor)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

7,307,564 training samples
811,952 validation samples


In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
torch.save(train_dataloader,path+"train_dataloader.pickle")
torch.save(validation_dataloader,path+"validation_dataloader.pickle")


In [ ]:
#testing
masked_index = 12
note_num = 0
sent = 1
note = data.input_ids_all[note_num]
retokenize = [tokenizer.convert_ids_to_tokens(sent) for sent in note]
real_sent = [tokenizer.convert_ids_to_tokens(sent) for sent in note]
retokenize[sent][masked_index] = '[MASK]'
start = retokenize[sent].index('[CLS]')+1
end = retokenize[sent].index('[SEP]')
retokenize = retokenize[sent][start:end]
print(retokenize)
encoded_dict = tokenizer.encode_plus(
                        retokenize,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
indexed_tokens = encoded_dict['input_ids']
attention_mask = encoded_dict['attention_mask']
print(indexed_tokens)

predictions = model(indexed_tokens, attention_mask = attention_mask)

predicted_index = torch.argmax(predictions[0][0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
print('predicted_token', predicted_token)
print("*"*80)
print(real_sent[sent])
print("*"*80)
print("*"*80)
print("*"*80)

for predic in predictions[0][0]:
    model.train()
    predicted_index = torch.argmax(predic).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print('predicted_token', predicted_token)
    print("*"*80)
    x+=1

In [ ]:
masked_index = 2
note_num = 0
sent = 1
note = data.input_ids_all[note_num]

predictions = model(inputa, attention_mask= att)
print(att)
x=0
for predic in predictions[0][0]:
    model.train()
    predicted_index = torch.argmax(predic).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print('predicted_token', predicted_token)
    print(real_sent[sent][x])
    print("*"*80)
    x+=1
